# Fathead Minnow

In [1]:
from helper_knn import *

db = pd.read_csv('lc_db_processed.csv').drop(columns = 'Unnamed: 0')

db = pd.concat([db,
                pd.DataFrame(pd.DataFrame(db['pubchem2d'].values).\
                             apply(lambda x: x.str.replace('', ' ').str.strip().str.split(' '), 
                                                                        axis = 1)[0].to_list(),
                   columns = ['pub'+ str(i) for i in range(1,882)])],
               axis = 1)

species = pd.read_csv('C:/Users/Simone/Desktop/Utilità tesi magistrale/data/species.txt', sep = '\|', engine = 'python')

fm = species[species['common_name'] == 'Fathead Minnow'][
    ['class', 'tax_order', 'family', 'genus', 'species']].values.ravel()

X_train = db.loc[(db[
    ['class', 'tax_order', 'family', 'genus', 'species']] == fm).all(axis =1)].copy().reset_index(drop=True)
y_train = np.where(X_train['conc1_mean'] > 1, 0,1)

X_test = db.loc[~(db[
    ['class', 'tax_order', 'family', 'genus', 'species']] == fm).all(axis = 1)].copy().reset_index(drop=True)
y_test = np.where(X_test['conc1_mean'] > 1, 0,1)

categorical = ['test_cas', 'fish', 'smiles', 'pubchem2d', 'conc1_mean', 'obs_duration_mean',
               'conc1_type', 'exposure_type', 'control_type', 'media_type',
               'application_freq_unit', 'species', 'class', 'tax_order', 'family', 'genus']
X_train.drop(columns = categorical, inplace = True)
X_test.drop(columns = categorical, inplace = True)

# KNN già fatto
file FM & RT

# Random forest

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import ParameterSampler

hyper_params_tune = {'max_depth' : [i for i in range(10,20,2)],
                     'n_estimators': [int(x) for x in np.linspace(start = 200, stop = 1000, num = 11)],
                     'min_samples_split': [2, 5, 10],
                     'min_samples_leaf': [1, 2, 4]}

params_comb = list(ParameterSampler(hyper_params_tune, n_iter = 100, random_state = 52))
best_acc = 0
best_sens = 0
best_spec = 0
best_f1 = 0
best_p = dict()
print(ctime())
for i in range(0, len(params_comb)):
    
    rfc = RandomForestClassifier(n_jobs = -1)

    for k,v in params_comb[i].items():
        setattr(rfc, k, v)
        
    rfc.fit(X_train, y_train)
    y_pred = rfc.predict(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    sens = recall_score(y_test, y_pred)
    
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    spec = tn/(tn+fp)
    f1 = f1_score(y_test, y_pred)
    
    if acc > best_acc:
        best_p = params_comb[i]
        best_sens = sens
        best_spec = spec
        best_acc = acc
        best_f1 = f1

print(ctime())
print('Model: ', best_p, "\n Accuracy: ", best_acc, "\n Sensitivity:", sens, "\n Specificity", best_spec, "\n F1 score:",
     best_f1)

Mon Feb 15 12:59:18 2021
Mon Feb 15 13:11:22 2021
Model:  {'n_estimators': 440, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_depth': 16} 
 Accuracy:  0.8240258989478553 
 Sensitivity: 0.6787835926449788 
 Specificity 0.9192412983965584 
 F1 score: 0.7612174458738625


# Simple RASAR

In [2]:
from sklearn.linear_model import LogisticRegression

def right_neighbor(neighbors, X_train, X_train_i):
    # IDX Neighbors
    idx_neigh_0 = pd.DataFrame(neighbors[1])[0].apply(lambda x: X_train_i.iloc[x].name)
    idx_neigh_1 = pd.DataFrame(neighbors[1])[1].apply(lambda x: X_train_i.iloc[x].name)
    
    idx_neigh = idx_neigh_0.copy()
    
    # dove l'indice del primo vicino risulta essere uguale a se stesso lo sostituisco con il secondo vicino
    idx_neigh[X_train.index == idx_neigh_0] = idx_neigh_1[X_train.index == idx_neigh_0].values
    
    # Distance from the Nearest Neighbor that is NOT itself
    dist_0 = pd.DataFrame(neighbors[0])[0]
    dist_1 = pd.DataFrame(neighbors[0])[1]
    
    distance = dist_0.copy()
    distance[X_train.index == idx_neigh_0] = dist_1[X_train.index == idx_neigh_0].values
    
    return idx_neigh, distance

In [4]:
X_train0 = X_train[y_train == 0].copy()
X_train1 = X_train[y_train == 1].copy()

print(ctime())
## DF Train

knn0 = KNeighborsClassifier(n_neighbors = 2)
knn0.fit(X_train0, y_train[y_train == 0])

neigh_train0 = knn0.kneighbors(X_train, return_distance = True)
idx_neigh0, dist0 = right_neighbor(neigh_train0, X_train, X_train0)

knn1 = KNeighborsClassifier(n_neighbors = 2)
knn1.fit(X_train1, y_train[y_train == 1])

neigh_train1 = knn1.kneighbors(X_train, return_distance = True)
idx_neigh1, dist1 = right_neighbor(neigh_train1, X_train, X_train1)

df_rasar_train = pd.DataFrame({'idx_train': X_train.index.values,
                      'label_train': y_train,
                      'idx_neigh0': idx_neigh0.values,
                      'dist_neigh0': dist0,       
                      'idx_neigh1': idx_neigh1.values,
                      'dist_neigh1': dist1})

# DF test
knn0 = KNeighborsClassifier(n_neighbors = 1)
knn0.fit(X_train0, y_train[y_train == 0])
neigh_test0 = knn0.kneighbors(X_test, return_distance = True)
idx_neigh_test0 = pd.DataFrame(neigh_test0[1])[0].apply(lambda x: X_train0.iloc[x].name)

knn1 = KNeighborsClassifier(n_neighbors = 1)
knn1.fit(X_train1, y_train[y_train == 1])
neigh_test1 = knn1.kneighbors(X_test, return_distance = True)
idx_neigh_test1 = pd.DataFrame(neigh_test1[1])[0].apply(lambda x: X_train1.iloc[x].name)

df_rasar_test = pd.DataFrame({'idx_test': X_test.index.values,
              'label_test': y_test,
              'idx_neigh0': idx_neigh_test0.values,
              'dist_neigh0': neigh_test0[0].ravel(),
              'idx_neigh1': idx_neigh_test1.values,
              'dist_neigh1': neigh_test1[0].ravel()})




lrc = LogisticRegression()
lrc.fit(df_rasar_train[['dist_neigh0','dist_neigh1']], y_train)
y_pred = lrc.predict(df_rasar_test[['dist_neigh0','dist_neigh1']])

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

print(ctime())
print("Accuracy: ", accuracy_score(y_test, y_pred),
      "\n Sensitivity:", recall_score(y_test, y_pred),
      "\n Specificity:", tn/(tn+fp),
     "\n F1 score:", f1_score(y_test, y_pred))

Mon Feb 15 13:13:42 2021
Mon Feb 15 13:14:26 2021
Accuracy:  0.7449416117470228 
 Sensitivity: 0.4514851485148515 
 Specificity: 0.9477903793508017 
 F1 score: 0.5913301222675065


# Rainbow Trout

In [1]:
from helper_knn import *

db = pd.read_csv('lc_db_processed.csv').drop(columns = 'Unnamed: 0')

db = pd.concat([db,
                pd.DataFrame(pd.DataFrame(db['pubchem2d'].values).\
                             apply(lambda x: x.str.replace('', ' ').str.strip().str.split(' '), 
                                                                        axis = 1)[0].to_list(),
                   columns = ['pub'+ str(i) for i in range(1,882)])],
               axis = 1)

species = pd.read_csv('C:/Users/Simone/Desktop/Utilità tesi magistrale/data/species.txt', sep = '\|', engine = 'python')

fm = species[species['common_name'] == 'Rainbow Trout'][
    ['class', 'tax_order', 'family', 'genus', 'species']].values.ravel()

X_train = db.loc[(db[
    ['class', 'tax_order', 'family', 'genus', 'species']] == fm).all(axis =1)].copy().reset_index(drop=True)
y_train = np.where(X_train['conc1_mean'] > 1, 0,1)

X_test = db.loc[~(db[
    ['class', 'tax_order', 'family', 'genus', 'species']] == fm).all(axis = 1)].copy().reset_index(drop=True)
y_test = np.where(X_test['conc1_mean'] > 1, 0,1)

categorical = ['test_cas', 'fish', 'smiles', 'pubchem2d', 'conc1_mean', 'obs_duration_mean',
               'conc1_type', 'exposure_type', 'control_type', 'media_type',
               'application_freq_unit', 'species', 'class', 'tax_order', 'family', 'genus']
X_train.drop(columns = categorical, inplace = True)
X_test.drop(columns = categorical, inplace = True)

# KNN già fatto

# Random forest

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import ParameterSampler

hyper_params_tune = {'max_depth' : [i for i in range(10,20,2)],
                     'n_estimators': [int(x) for x in np.linspace(start = 200, stop = 1000, num = 11)],
                     'min_samples_split': [2, 5, 10],
                     'min_samples_leaf': [1, 2, 4]}

params_comb = list(ParameterSampler(hyper_params_tune, n_iter = 100, random_state = 52))
best_acc = 0
best_sens = 0
best_spec = 0
best_f1 = 0
best_p = dict()
print(ctime())
for i in range(0, len(params_comb)):
    
    rfc = RandomForestClassifier(n_jobs = -1)

    for k,v in params_comb[i].items():
        setattr(rfc, k, v)
        
    rfc.fit(X_train, y_train)
    y_pred = rfc.predict(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    sens = recall_score(y_test, y_pred)
    
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    spec = tn/(tn+fp)
    f1 = f1_score(y_test, y_pred)
    
    if acc > best_acc:
        best_p = params_comb[i]
        best_sens = sens
        best_spec = spec
        best_acc = acc
        best_f1 = f1

print(ctime())
print('Model: ', best_p, "\n Accuracy: ", best_acc, "\n Sensitivity:", sens, "\n Specificity", best_spec, "\n F1 score:",
     best_f1)

Mon Feb 15 13:18:20 2021
Mon Feb 15 13:28:15 2021
Model:  {'n_estimators': 520, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': 12} 
 Accuracy:  0.8446083995459704 
 Sensitivity: 0.8307276007215875 
 Specificity 0.8438183807439825 
 F1 score: 0.8043167524299599


# Simple RASAR

In [3]:
X_train0 = X_train[y_train == 0].copy()
X_train1 = X_train[y_train == 1].copy()

print(ctime())
## DF Train

knn0 = KNeighborsClassifier(n_neighbors = 2)
knn0.fit(X_train0, y_train[y_train == 0])

neigh_train0 = knn0.kneighbors(X_train, return_distance = True)
idx_neigh0, dist0 = right_neighbor(neigh_train0, X_train, X_train0)

knn1 = KNeighborsClassifier(n_neighbors = 2)
knn1.fit(X_train1, y_train[y_train == 1])

neigh_train1 = knn1.kneighbors(X_train, return_distance = True)
idx_neigh1, dist1 = right_neighbor(neigh_train1, X_train, X_train1)

df_rasar_train = pd.DataFrame({'idx_train': X_train.index.values,
                      'label_train': y_train,
                      'idx_neigh0': idx_neigh0.values,
                      'dist_neigh0': dist0,       
                      'idx_neigh1': idx_neigh1.values,
                      'dist_neigh1': dist1})

# DF test
knn0 = KNeighborsClassifier(n_neighbors = 1)
knn0.fit(X_train0, y_train[y_train == 0])
neigh_test0 = knn0.kneighbors(X_test, return_distance = True)
idx_neigh_test0 = pd.DataFrame(neigh_test0[1])[0].apply(lambda x: X_train0.iloc[x].name)

knn1 = KNeighborsClassifier(n_neighbors = 1)
knn1.fit(X_train1, y_train[y_train == 1])
neigh_test1 = knn1.kneighbors(X_test, return_distance = True)
idx_neigh_test1 = pd.DataFrame(neigh_test1[1])[0].apply(lambda x: X_train1.iloc[x].name)

df_rasar_test = pd.DataFrame({'idx_test': X_test.index.values,
              'label_test': y_test,
              'idx_neigh0': idx_neigh_test0.values,
              'dist_neigh0': neigh_test0[0].ravel(),
              'idx_neigh1': idx_neigh_test1.values,
              'dist_neigh1': neigh_test1[0].ravel()})




lrc = LogisticRegression()
lrc.fit(df_rasar_train[['dist_neigh0','dist_neigh1']], y_train)
y_pred = lrc.predict(df_rasar_test[['dist_neigh0','dist_neigh1']])

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

print(ctime())
print("Accuracy: ", accuracy_score(y_test, y_pred),
      "\n Sensitivity:", recall_score(y_test, y_pred),
      "\n Specificity:", tn/(tn+fp),
     "\n F1 score:", f1_score(y_test, y_pred))

Mon Feb 15 13:16:30 2021
Mon Feb 15 13:17:14 2021
Accuracy:  0.8043700340522134 
 Sensitivity: 0.685207456404089 
 Specificity: 0.8766411378555798 
 F1 score: 0.7256228607816604
